# JSONLD Wrapper

In [1]:
import json
from pyld import jsonld

## Wrapper
Can a wrapper to handle interoperability easier.
How the wrapper works:

```json
{
   "vc":{
      "valid jsonld"
   },
   "req":{
      "The requirements for a VC to be considered valid, including its required fields, 
      are specified under the 'spec' format. When mapping is done via 'spec', it is 
      possible for duplications to occur from the 'VC'."
   },
   "spec":{
      "wrapper-for-what-vc"
   }
}
```

## EU-VC-LD
Possible working name:
EU Verifiable Credential JSON-LD (EU-VC-LD)

In [2]:
# Creates the input to the _sd
class EUVCLD:
    def __init__(self):
        self.euvc = None

    def create_euvcld(self, ctx, required, spec=''):
        if 'vc' in ctx:
            v = ctx
        else:
            v = {"vc": ctx}

        if 'req' in required:
            r = required
        else:
            r = {"req": required}

        if 'spec' in spec:
            s = spec
        elif isinstance(spec, str) and len(spec)>0:
            s = {"spec": {
                "type": spec,
                "version": "1",
            }}
        else:
            s = {}

        self.euvc = {**v, **r, **s}
        return self.euvc

    # create JSONLD from EUVCLD
    def create_jsonld(self, keepVC=False):
        if keepVC:
            v = {"vc": self.euvc['vc']}
        else:
            v = self.euvc['vc']

        return {**v, **self.euvc['req']}

    # some sanity checks
    def valid_spec(self, ctx, required, spec):
        # need later do some checks
        return True
        


## Examples
Different challanges with vc were wrapper is used:

### Challange 1: VC-JWT
- https://www.w3.org/TR/vc-data-model-2.0/#example-usage-of-the-id-property

In [3]:
v1 = {
    "@context": [
        "https://www.w3.org/ns/credentials/v2",
        "https://www.w3.org/ns/credentials/examples/v2"
    ],
    "id": "http://university.example/credentials/3732",
    "type": ["VerifiableCredential", "ExampleDegreeCredential"],
    "issuer": "https://university.example/issuers/565049",
    "validFrom": "2010-01-01T00:00:00Z",
    "credentialSubject": {
        "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
        "degree": {
            "type": "ExampleBachelorDegree",
            "name": "Bachelor of Science and Arts"
        }
    }
}

r1 = {
  "iss": "https://university.example/issuers/565049",
  "jti": "http://university.example/credentials/3732",
  "sub": "did:example:ebfeb1f712ebc6f1c276e12ec21"
}

In [4]:
# init the wrapper vc
evc1 = EUVCLD()
# create vc
evc1.create_euvcld(v1, r1, 'vc-jwt')
# print result
print(json.dumps(evc1.euvc, indent=2))
# check valid jsonld
assert(jsonld.expand(evc1.create_jsonld()))

{
  "vc": {
    "@context": [
      "https://www.w3.org/ns/credentials/v2",
      "https://www.w3.org/ns/credentials/examples/v2"
    ],
    "id": "http://university.example/credentials/3732",
    "type": [
      "VerifiableCredential",
      "ExampleDegreeCredential"
    ],
    "issuer": "https://university.example/issuers/565049",
    "validFrom": "2010-01-01T00:00:00Z",
    "credentialSubject": {
      "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
      "degree": {
        "type": "ExampleBachelorDegree",
        "name": "Bachelor of Science and Arts"
      }
    }
  },
  "req": {
    "iss": "https://university.example/issuers/565049",
    "jti": "http://university.example/credentials/3732",
    "sub": "did:example:ebfeb1f712ebc6f1c276e12ec21"
  },
  "spec": {
    "type": "vc-jwt",
    "version": "1"
  }
}


Result from W3C

In [5]:
res1 = {
  "vc": {
    "@context": [
      "https://www.w3.org/ns/credentials/v2",
      "https://www.w3.org/ns/credentials/examples/v2"
    ],
    "id": "http://university.example/credentials/3732",
    "type": [
      "VerifiableCredential",
      "ExampleDegreeCredential"
    ],
    "issuer": "https://university.example/issuers/565049",
    "validFrom": "2010-01-01T00:00:00Z",
    "credentialSubject": {
      "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
      "degree": {
        "type": "ExampleBachelorDegree",
        "name": "Bachelor of Science and Arts"
      }
    }
  },
  "iss": "https://university.example/issuers/565049",
  "jti": "http://university.example/credentials/3732",
  "sub": "did:example:ebfeb1f712ebc6f1c276e12ec21"
}

Verify result is equal wrapper

In [6]:
# convert to jsonld and verify if the result is correct
assert(evc1.create_jsonld(keepVC=True) == res1)

### Challange 2: SD-JWT
- https://www.w3.org/TR/vc-jose-cose/#selective-disclosure
- https://datatracker.ietf.org/doc/html/draft-ietf-oauth-selective-disclosure-jwt-05#name-example-4b-w3c-verifiable-c

In [7]:
v2 = {
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://w3id.org/vaccination/v1"
  ],
  "type": [
    "VerifiableCredential",
    "VaccinationCertificate"
  ],
  "issuer": "https://example.com/issuer",
  "issuanceDate": "2023-02-09T11:01:59Z",
  "expirationDate": "2028-02-08T11:01:59Z",
  "name": "COVID-19 Vaccination Certificate",
  "description": "COVID-19 Vaccination Certificate",
  "credentialSubject": {
    "_sd": [
      "1V_K-8lDQ8iFXBFXbZY9ehqR4HabWCi5T0ybIzZPeww",
      "JzjLgtP29dP-B3td12P674gFmK2zy81HMtBgf6CJNWg",
      "R2fGbfA07Z_YlkqmNZyma1xyyx1XstIiS6B1Ybl2JZ4",
      "TCmzrl7K2gev_du7pcMIyzRLHp-Yeg-Fl_cxtrUvPxg",
      "V7kJBLK78TmVDOmrfJ7ZuUPHuK_2cc7yZRa4qV1txwM",
      "b0eUsvGP-ODDdFoY4NlzlXc3tDslWJtCJF75Nw8Oj_g",
      "zJK_eSMXjwM8dXmMZLnI8FGM08zJ3_ubGeEMJ-5TBy0"
    ],
    "vaccine": {
      "_sd": [
        "1cF5hLwkhMNIaqfWJrXI7NMWedL-9f6Y2PA52yPjSZI",
        "Hiy6WWueLD5bn16298tPv7GXhmldMDOTnBi-CZbphNo",
        "Lb027q691jXXl-jC73vi8ebOj9smx3C-_og7gA4TBQE"
      ],
      "type": "Vaccine"
    },
    "recipient": {
      "_sd": [
        "1lSQBNY24q0Th6OGzthq-7-4l6cAaxrYXOGZpeW_lnA",
        "3nzLq81M2oN06wdv1shHvOEJVxZ5KLmdDkHEDJABWEI",
        "Pn1sWi06G4LJrnn-_RT0RbM_HTdxnPJQuX2fzWv_JOU",
        "lF9uzdsw7HplGLc714Tr4WO7MGJza7tt7QFleCX4Itw"
      ],
      "type": "VaccineRecipient"
    },
    "type": "VaccinationEvent"
  }
}

r2 = {
  "iss": "https://example.com/issuer",
  "iat": 1683000000,
  "exp": 1883000000,
  "_sd_alg": "sha-256"
}

In [8]:
# init the wrapper vc
evc2 = EUVCLD()
# create vc
evc2.create_euvcld(v2, r2, 'sd-jwt')
# print result
print(json.dumps(evc2.euvc, indent=2))
# check valid jsonld
assert(jsonld.expand(evc2.create_jsonld()))

{
  "vc": {
    "@context": [
      "https://www.w3.org/2018/credentials/v1",
      "https://w3id.org/vaccination/v1"
    ],
    "type": [
      "VerifiableCredential",
      "VaccinationCertificate"
    ],
    "issuer": "https://example.com/issuer",
    "issuanceDate": "2023-02-09T11:01:59Z",
    "expirationDate": "2028-02-08T11:01:59Z",
    "name": "COVID-19 Vaccination Certificate",
    "description": "COVID-19 Vaccination Certificate",
    "credentialSubject": {
      "_sd": [
        "1V_K-8lDQ8iFXBFXbZY9ehqR4HabWCi5T0ybIzZPeww",
        "JzjLgtP29dP-B3td12P674gFmK2zy81HMtBgf6CJNWg",
        "R2fGbfA07Z_YlkqmNZyma1xyyx1XstIiS6B1Ybl2JZ4",
        "TCmzrl7K2gev_du7pcMIyzRLHp-Yeg-Fl_cxtrUvPxg",
        "V7kJBLK78TmVDOmrfJ7ZuUPHuK_2cc7yZRa4qV1txwM",
        "b0eUsvGP-ODDdFoY4NlzlXc3tDslWJtCJF75Nw8Oj_g",
        "zJK_eSMXjwM8dXmMZLnI8FGM08zJ3_ubGeEMJ-5TBy0"
      ],
      "vaccine": {
        "_sd": [
          "1cF5hLwkhMNIaqfWJrXI7NMWedL-9f6Y2PA52yPjSZI",
          "Hiy6WWueLD5bn1629

Result from SD-JWT

In [9]:
res2 = {
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://w3id.org/vaccination/v1"
  ],
  "type": [
    "VerifiableCredential",
    "VaccinationCertificate"
  ],
  "issuer": "https://example.com/issuer",
  "issuanceDate": "2023-02-09T11:01:59Z",
  "expirationDate": "2028-02-08T11:01:59Z",
  "name": "COVID-19 Vaccination Certificate",
  "description": "COVID-19 Vaccination Certificate",
  "credentialSubject": {
    "_sd": [
      "1V_K-8lDQ8iFXBFXbZY9ehqR4HabWCi5T0ybIzZPeww",
      "JzjLgtP29dP-B3td12P674gFmK2zy81HMtBgf6CJNWg",
      "R2fGbfA07Z_YlkqmNZyma1xyyx1XstIiS6B1Ybl2JZ4",
      "TCmzrl7K2gev_du7pcMIyzRLHp-Yeg-Fl_cxtrUvPxg",
      "V7kJBLK78TmVDOmrfJ7ZuUPHuK_2cc7yZRa4qV1txwM",
      "b0eUsvGP-ODDdFoY4NlzlXc3tDslWJtCJF75Nw8Oj_g",
      "zJK_eSMXjwM8dXmMZLnI8FGM08zJ3_ubGeEMJ-5TBy0"
    ],
    "vaccine": {
      "_sd": [
        "1cF5hLwkhMNIaqfWJrXI7NMWedL-9f6Y2PA52yPjSZI",
        "Hiy6WWueLD5bn16298tPv7GXhmldMDOTnBi-CZbphNo",
        "Lb027q691jXXl-jC73vi8ebOj9smx3C-_og7gA4TBQE"
      ],
      "type": "Vaccine"
    },
    "recipient": {
      "_sd": [
        "1lSQBNY24q0Th6OGzthq-7-4l6cAaxrYXOGZpeW_lnA",
        "3nzLq81M2oN06wdv1shHvOEJVxZ5KLmdDkHEDJABWEI",
        "Pn1sWi06G4LJrnn-_RT0RbM_HTdxnPJQuX2fzWv_JOU",
        "lF9uzdsw7HplGLc714Tr4WO7MGJza7tt7QFleCX4Itw"
      ],
      "type": "VaccineRecipient"
    },
    "type": "VaccinationEvent"
  },
  "iss": "https://example.com/issuer",
  "iat": 1683000000,
  "exp": 1883000000,
  "_sd_alg": "sha-256"
}

Verify result is equal wrapper

In [10]:
# convert to jsonld and verify if the result is correct
assert(evc2.create_jsonld(keepVC=False) == res2)